In [1]:
# import libraries

import boto3, pandas as pd

from time import sleep

In [2]:
# create client to AWS Athena

client = boto3.client('athena')

# function to build query

def athena_query(sql, output_location):

    response = client.start_query_execution(
        QueryString = sql,
        QueryExecutionContext = {'Database': 'edc_mod1_desafio_database'}, 
        ResultConfiguration = {'OutputLocation': output_location}
    )

    query_status = []

    while not query_status == 'SUCCEEDED':
        query_status= client.get_query_execution(QueryExecutionId=response["QueryExecutionId"])
        query_status = query_status["QueryExecution"]["Status"]["State"]
        sleep(2)

    return response["QueryExecutionId"]

# function to read query results

def query_results(query_id):

    response = client.get_query_results(
        QueryExecutionId=query_id
    )

    return response

# function to convert query results to dataframe

def results_to_df(query_results):
 
    columns = [
        col['Label']
        for col in query_results['ResultSet']['ResultSetMetadata']['ColumnInfo']
    ]
 
    listed_results = []

    for res in query_results['ResultSet']['Rows'][1:]:         
        values = []

        for field in res['Data']:
            try:
                values.append(list(field.values())[0]) 
            except:
                values.append(list(' '))
 
        listed_results.append(
            dict(zip(columns, values))
        )
 
    return pd.DataFrame(listed_results)

In [ ]:
# set query outut location

output_location = 's3://datalake-igti-fabio-rais/athena-results/'

In [9]:
# Pergunta 1: Qual é o SEGUNDO motivo de desligamento mais frequente?

sql = ('''

        SELECT
            motivo_desligamento,
            COUNT(*) as qtd
            
        FROM staging_zone

        GROUP BY motivo_desligamento

        ORDER BY 2 DESC

        LIMIT 10

    ''')

df = results_to_df(query_results(athena_query(sql=sql, output_location=output_location)))
df

,motivo_desligamento,qtd
0,0,46245313
1,11,9771002
2,21,4097890
3,12,3606086
4,31,1439716
5,10,228301
6,90,186095
7,60,89003
8,70,61576
9,30,46149


In [10]:
# Pergunta 2: Qual é o estado com a maior média de renda nominal do Brasil?

sql = ('''

        SELECT
            AVG(vl_remun_media_nom) as renda_media,
            uf
            
        FROM staging_zone

        GROUP BY uf

        ORDER BY 1 DESC

        LIMIT 10

    ''')

df = results_to_df(query_results(athena_query(sql=sql, output_location=output_location)))
df

,renda_media,uf
0,4454.921351990675,53
1,3403.5848835592583,16
2,2976.293270263938,33
3,2854.950994086181,35
4,2748.4508343915277,14
5,2658.4066067570147,12
6,2532.6492588971673,13
7,2524.5165638868007,43
8,2510.981523246122,17
9,2420.9441663880516,51


In [12]:
# Pergunta 3: Qual é a média da renda nominal da UF 31 (utilize a renda média nominal)?

sql = ('''

        SELECT
            AVG(vl_remun_media_nom) as renda_media,
            uf
            
        FROM staging_zone

        WHERE uf = 31

        GROUP BY uf

        ORDER BY 1 DESC

    ''')

df = results_to_df(query_results(athena_query(sql=sql, output_location=output_location)))
df

,renda_media,uf
0,2220.012129816938,31


In [14]:
# Pergunta 4: Quantas pessoas, no estado 21, possuem renda nominal média menor do que o salário-mínimo de 2020 (1039 reais)?

sql = ('''

        SELECT
            uf,
            COUNT(*)
            
        FROM staging_zone

        WHERE 
            uf = 21 AND
            vl_remun_media_nom < 1039

        GROUP BY uf

        ORDER BY 1 DESC

    ''')

df = results_to_df(query_results(athena_query(sql=sql, output_location=output_location)))
df

,uf,_col1
0,21,155105


In [15]:
# Pergunta 5: Qual foi a média de horas trabalhadas na UF 16?

sql = ('''

        SELECT
            uf,
            avg(qtd_hora_contr) as horas_trabalhas
            
        FROM staging_zone

        WHERE 
            uf = 16

        GROUP BY uf

        ORDER BY 1 DESC

    ''')

df = results_to_df(query_results(athena_query(sql=sql, output_location=output_location)))
df

,uf,horas_trabalhas
0,16,39.539764150049585


In [16]:
# Pergunta 6: Qual é a diferença entre os salários médios nominais entre categorias de sexo?

sql = ('''

        SELECT
            sexo_trabalhador,
            avg(vl_remun_media_nom) as renda_media
            
        FROM staging_zone

        GROUP BY sexo_trabalhador

        ORDER BY 1 DESC

    ''')

df = results_to_df(query_results(athena_query(sql=sql, output_location=output_location)))
df

,sexo_trabalhador,renda_media
0,2.0,2396.9956602075818
1,1.0,2705.7238064088015


In [17]:
# Pergunta 7: Qual é a diferença entre os salários médios nominais entre categorias de sexo para os trabalhadores da área de tecnologia (CNAE 2.0 Classe = 62040)?

sql = ('''

        SELECT
            sexo_trabalhador,
            avg(vl_remun_media_nom) as renda_media
            
        FROM staging_zone

        WHERE cnae_2_0_classe = 62040

        GROUP BY sexo_trabalhador

        ORDER BY 1 DESC

    ''')

df = results_to_df(query_results(athena_query(sql=sql, output_location=output_location)))
df

,sexo_trabalhador,renda_media
0,2.0,4182.351458541008
1,1.0,5989.4694159002065


In [5]:
df.columns

Index(['bairros_sp', 'bairros_fortaleza', 'bairros_rj', 'causa_afastamento_1',
       'causa_afastamento_2', 'causa_afastamento_3', 'motivo_desligamento',
       'cbo_ocupacao_2002', 'cnae_2_0_classe', 'cnae_95_classe',
       'distritos_sp', 'vinculo_ativo_31_12', 'faixa_etaria',
       'faixa_hora_contrat', 'faixa_remun_dezem_sm', 'faixa_remun_media_sm',
       'faixa_tempo_emprego', 'escolaridade_apos_2005', 'qtd_hora_contr',
       'idade', 'ind_cei_vinculado', 'ind_simples', 'mes_admissao',
       'mes_desligamento', 'mun_trab', 'municipio', 'nacionalidade',
       'natureza_juridica', 'ind_portador_defic', 'qtd_dias_afastamento',
       'raca_cor', 'regioes_adm_df', 'vl_remun_dezembro_nom',
       'vl_remun_dezembro_sm', 'vl_remun_media_nom', 'vl_remun_media_sm',
       'cnae_2_0_subclasse', 'sexo_trabalhador', 'tamanho_estabelecimento',
       'tempo_emprego', 'tipo_admissao', 'tipo_estab41', 'tipo_estab42',
       'tipo_defic', 'tipo_vinculo', 'ibge_subsetor', 'vl_rem_janeiro_s